We can establish our framework by defining the variables to be used.  We start with the outcome variable, $Y$.  Next, we consider out treatment variable, $A_k$, which takes on value 1 if there is treatment, and is 0 otherwise.  To begin, we will use treatment strategies that are exclusively treatment or exclusively no treatment, corresponding to $\overline{a} = (1,1,\dots,1) = \overline{1}$ and $\overline{a} = (0,0,\dots,0) = \overline{0}$ respectively.  The next measurable variable is $L$, which represents the covariate(s) to be included.  Note that both the covariates, $L$, and the outcome, $Y$ are affected by an unmeasured common cause, $U$.  The diagram below illustrates these relationships.  

![title](image4.png)

The purpose of this investigation is to measure the average causal effect of treatment, which can be estimated using 
$$ \mathbb{E}\big[Y^{a=1}\big] - \mathbb{E}\big[Y^{a=0}\big]$$ 

We want to build out the g-formula as follows  
$$ \mathbb{E} \big[Y_{i}^{\overline{a}}\big]  = \sum_{i} \mathbb{E} \big[Y_i \mid A_{i-1} = a_{i-1},  \; A_{i-2} = a_{i-2}, \; L_{i-1}, \; L_{i-2} \big]$$ 

We can do this by building out two models, one for $Y$ And one for $L$.  We will begin by using a continous $Y$ and a binary $L$ for simplicity.  

For $Y$, we will use a linear regression for a time delay of t=2.  The model will look something like this 

$$\mathbb{E} \big[Y_{t+1} \mid \overline{A}_t, \overline{L}_t, \overline{Y}_t \big] = \theta_{0,t} + \theta_1 Y_{t} + \theta_2 Y_{t-1} + \theta_3 A_{t}+ \theta_4 A_{t-1} + \theta_5 L_t + \theta_6 L_{t-1} $$ 

For each $L$, we will use a logistic regression, also calculated on a time delay of t=2.  This will give us something similar to 
$$ logistP\big[L_{t+1} \mid \overline{A}_t, \overline{L}_t, \overline{Y}_{t+1} \big] = \alpha_{0,t} + \alpha_1 Y_{t+1} + \alpha_2 Y_{t} + \alpha_3 Y_{t-1} +  \alpha_4 A_{t}+ \alpha_5 A_{t-1} + \alpha_6 L_t + \alpha_7 L_{t-1} $$ 


<!---what quadratic time term needed to be added here? --->

We first need to simulate the data. We will build out the covariates of the population.  For simplicity, we will use one binary covariate, $L_1$.  

The data is being simulated using the following equations, where U is an underlying confounder.  

$$logitP[L_k] = \alpha_0 + \alpha_1 \cdot L_{k-1} + \alpha_2 \cdot L_{k-2} + \alpha_3 A_{k-1} + \alpha_4 A_{K-2} + \alpha_5 U$$ 


$$ logit[A_k] = \beta_0 + \beta_1 L_{k} + \beta_2 L_{k-1} + \beta_3 A_{k-1} + \beta_4 A_{K-2} $$ 

Then, an end $Y$ value will be pulled from the following 
$$ Y \sim N(\mu = U, \sigma = 1) $$ 

In [383]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
import math
import csv
import statsmodels.api as sm
import statsmodels.formula.api as smf
import random
import time 

In [364]:
#########################################################################
##[FUNCTION] data_creation simulates data for a given number of 
## individuals(indiv) over a set amount of time (max_time), and can 
## include as many covariates as desired (number_of_covariates)

## -- need to create the functionality for multiple covariates

#########################################################################


def data_creation(indiv, max_time, number_of_covariates): 

    columns = ["indiv", "time","U", "A", "Y",  "L1"]
    df = pd.DataFrame(columns = columns)
    coefficient_df = pd.DataFrame(columns = ["indiv", "alpha_0", "alpha_1", \
                     "alpha_2", "alpha_3", "alpha_4", "alpha_5", "beta_0", \
                     "beta_1", "beta_2", "beta_3", "beta_4"])

    for ii in range(1,indiv+1):

        alpha_0 = np.random.uniform(low = -1, high = 1)
        alpha_1 = np.random.uniform(low = -1, high = 1)
        alpha_2 = np.random.uniform(low = -1, high = 1)
        alpha_3 = np.random.uniform(low = -1, high = 1)
        alpha_4 = np.random.uniform(low = -1, high = 1)
        alpha_5 = np.random.uniform(low = -1, high = 1)

        beta_0 = np.random.uniform(low = -1, high = 1)
        beta_1 = np.random.uniform(low = -1, high = 1)
        beta_2 = np.random.uniform(low = -1, high = 1)
        beta_3 = np.random.uniform(low = -1, high = 1)
        beta_4 = np.random.uniform(low = -1, high = 1)

        coefficient_df.loc[len(coefficient_df)+1] = [ii, alpha_0, alpha_1, alpha_2, \
                alpha_3, alpha_4, alpha_5, beta_0, beta_1, beta_2, beta_3, beta_4, ]

        for jj in range(0, max_time+1): 

            ## creating an unobserved variable that affects covariates 
            U = np.random.uniform(low = 0.1, high = 1)

            if jj == 0: 
                x_L = alpha_0 + alpha_5*U 
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))

                x_A = beta_0 + beta_1*L1 
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                df.loc[len(df)+1] = [ii, jj, U, A, "NaN",  L1]

            elif jj == 1: 
                x_L = alpha_0 + alpha_1*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)]) \
                    +alpha_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])+ alpha_5*U 
                
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))

                
                x_A = beta_0 + beta_1*L1 + beta_2*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    + beta_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])
                
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                df.loc[len(df)+1] = [ii, jj, U, A, "NaN", L1]

            else: 
                x_L = alpha_0 + alpha_1*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    +alpha_2*float(df["L1"][(df.time == jj-2) & (df.indiv == ii)])\
                    +alpha_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])\
                    +alpha_4*float(df["A"][(df.time == jj-2) & (df.indiv == ii)])+ alpha_5*U 
                
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))


                x_A = beta_0 + beta_1*L1 + beta_2*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    + beta_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])+ \
                    beta_4*float(df["A"][(df.time == jj-2) & (df.indiv == ii)])
                
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                if jj == max_time: 
                    Y = np.random.normal(loc = U, scale = 1)
                    df.loc[len(df)+1] = [ii, jj, U, A, Y, L1]

                else: 
                    df.loc[len(df)+1] = [ii, jj, U, A, "NaN", L1]

    # creating shifted values 
    for kk in range(1,max_time+1):
        df["L1_"+str(kk)] = df.L1.shift(kk)
        df["A_"+str(kk)] = df.A.shift(kk)

    return(df, coefficient_df); 

Starting with the model for Y, we use a linear regression to get the following model, 
$$\mathbb{E} \big[Y \mid \overline{A}_t, \overline{L}_t \big] = \theta_{0,t} + \theta_1 A_{t}+ \theta_2 A_{t-1} + \theta_3 L_t + \theta_4 L_{t-1} $$ 

In [365]:
## Creating model for Y 

def Y_model_creation(df, max_time): 
    temp_df = df[df.time == max_time]
    train_columns ='+'.join(map(str, np.append(list(df)[3],list(df)[5:])))
    temp_df = temp_df.astype(float)
    Y_model = smf.ols("Y~"+train_columns, data=temp_df).fit()
    return(Y_model)

In [372]:
## Creating model for L1

def covariate_model_creation(df, max_time): 
    columns = ["time", "gamma_0", "gamma_1", "gamma_2", "gamma_3", "gamma_4"]
    train_columns = ["L1_1", "L1_2", "A_1", "A_2"]
    L1_model_df = pd.DataFrame(columns = columns)

    for ii in range(1, (max_time+1)): 
        temp_df = df[df.time == ii]   
        if ii == 1: 
            L1_model = sm.Logit(np.asarray(temp_df["L1"]), np.asarray(sm.add_constant(temp_df[["L1_1", "A_1"]]))).fit()
            L1_model_df = L1_model_df.append(pd.DataFrame([ii] + [L1_model.params[i] for i in range(0,2)] + ["Nan"] + [L1_model.params[2]] + ["Nan"], index = columns).transpose(), ignore_index=True)
        else: 
            L1_model = sm.Logit(np.asarray(temp_df["L1"]), np.asarray(sm.add_constant(temp_df[train_columns]))).fit()
            L1_model_df = L1_model_df.append(pd.DataFrame([ii] + [L1_model.params[i] for i in range(0,5)], index = columns).transpose(), ignore_index=True)
    return(L1_model_df)

## Running the Monte Carlo Simulation
Now, using these three models, we can calculate the treatment effect of 

$$ \mathbb{E}\big[Y^{a=1}\big] - \mathbb{E}\big[Y^{a=0}\big]$$ 

where 

$$ \mathbb{E} \big[Y^{\overline{a}}\big]  = \sum_{i} \mathbb{E} \big[Y \mid A_{i-1} = a_{i-1},  \; A_{i-2} = a_{i-2}, \; L_{1, i-1}, \; L_{1, i-2},\big] \\ 
= \int_{l_i}  $$

for the respective $\bar{a} = \bar{0}$ and $\bar{a} = \bar{1}$

In [384]:
def simulation_run(df, Y_model, L1_model_df, max_time): 
    reps = 10000
    final_results_1 = np.empty(reps) 

    ### establishing treatment of interest
    A_test = [1]*max_time

    for ii in range(0,reps):
        values = np.empty(max_time)
        values[0] = random.choice(list(df["L1"][df["time"] == 0]))
        if values[0] == 0: 
            prod = 1-np.mean(list(df["L1"][df["time"] == 0]))
        else: 
            prod = np.mean(list(df["L1"][df["time"] == 0]))

        for jj in range(1, max_time):
            if jj == 1: 
                values[1] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in [1,2,4]])*[1,values[jj-1],A_test[jj-1]])
            else: 
                values[jj] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in range(1,6)])*[1,values[jj-1],values[jj-2], A_test[jj-1], A_test[jj-2]])
            prod = prod*(np.exp(values[jj])/(1+np.exp(values[jj])))

        list1 = [A_test[max_time-i] for i in range(1,max_time+2)]
        list2 = [values[max_time-i] for i in range(1,max_time+2)]

        result = [None]*(len(list1)+len(list2))
        result[::2] = list1
        result[1::2] = list2
        result = [1] + result

        Y_exp = np.sum(np.array(Y_model.params)*result)

        final_results_1[ii] = prod*Y_exp

    mean_1 = np.mean(final_results_1)


    final_results_0 = np.empty(reps) 
    A_test = [0]*max_time

    for ii in range(0,reps):
        values = np.empty(max_time)
        values[0] = random.choice(list(df["L1"][df["time"] == 0]))
        if values[0] == 0: 
            prod = 1-np.mean(list(df["L1"][df["time"] == 0]))
        else: 
            prod = np.mean(list(df["L1"][df["time"] == 0]))

        for jj in range(1, max_time):
            if jj == 1: 
                values[1] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in [1,2,4]])*[1,values[jj-1],A_test[jj-1]])
            else: 
                values[jj] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in range(1,6)])*[1,values[jj-1],values[jj-2], A_test[jj-1], A_test[jj-2]])
            prod = prod*(np.exp(values[jj])/(1+np.exp(values[jj])))

        list1 = [A_test[max_time-i] for i in range(1,max_time+2)]
        list2 = [values[max_time-i] for i in range(1,max_time+2)]

        result = [None]*(len(list1)+len(list2))
        result[::2] = list1
        result[1::2] = list2
        result = [1] + result

        Y_exp = np.sum(np.array(Y_model.params)*result)

        final_results_0[ii] = prod*Y_exp

    mean_0 = np.mean(final_results_0)
    final_answer = mean_1 - mean_0 
    return(final_answer) 

## Simulation Study


In [386]:
## establishing constants 
indiv = 500   ## number of individuals in study 
max_time = 10 ## number of time points being considered 
t_delay = 2 ## number of time delays included in model 
num_sims = 500 
results = np.empty(num_sims)

## RUNNING SIMULATIONS 
start_time = time.time() 
for ii in range(num_sims):
    print(ii) 
    [df, coefficient_df] = data_creation(indiv,max_time, 2) 
    Y_model = Y_model_creation(df, max_time)
    L1_model_df = covariate_model_creation(df, max_time)
    results[ii] = simulation_run(df, Y_model, L1_model_df, max_time)

elapsed_time = time.time() - start_time

0
Optimization terminated successfully.
         Current function value: 0.678842
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684739
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.664209
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.666502
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.655964
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.671642
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.662488
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.658271
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.673060
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.657280


In [394]:
from tempfile import TemporaryFile
outfile = TemporaryFile()
np.save(outfile, results)

In [392]:
np.mean(results)

1.7217286172967964e-07

In [393]:
np.var(results)

2.3894070465421745e-10

## FINAL RESULTS 
mean of means of differences: 1.7217286172967964e-07

variance of means of differences: 2.3894070465421745e-10